In [ ]:
import tensorflow as tf
from os import listdir, mkdir, system
from os.path import isfile, join
import shutil
from progressbar import ProgressBar
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
names=["00010", 
       "00012", 
       "00015",
       "00019",
       "00023",
       "00028",
       "00034",
       "00042",
       "00052",
       "00064",
       "00078",
       "00096",
       "00118",
       "00145",
       "00179",
       "00219",
       "00270",
       "00331",
       "00407",
       "00500"]

upper_limit=[0.0011, 
             0.0013, 
             0.0017, 
             0.0021, 
             0.0025, 
             0.0031, 
             0.0038, 
             0.0047, 
             0.0058, 
             0.0071, 
             0.0087,  
             0.0101,   
             0.0131,   
             0.0162,   
             0.0199,   
             0.0244,   
             0.0300,   
             0.0369,   
             0.0453]

upper_limit_ALT=[11, 
                 13, 
                 17, 
                 21, 
                 25, 
                 31, 
                 38, 
                 47, 
                 58, 
                 71, 
                 87,  
                 101,   
                 131,   
                 162,   
                 199,   
                 244,   
                 300,   
                 369,   
                 453]

print("Using",len(names),"discrete classes")
print("With",len(upper_limit), "upper limits assigned")
print("highest class is treated as un-bounded")

In [ ]:
dataset_location = "I:/WOLO/BENCHMARK/MultiCamAnts_sub-datasets/PLAIN_DSLR_patches"
output_location = "I:/WOLO/BENCHMARK/MultiCamAnts"
pbar = ProgressBar()
make_test_train = True
startframe = 5754
num_train = 8000

SPLIT_KEY = "_"
DATA_FORMAT = "jpg"
USE_CLASS_LIMITS = True
OUTPUT_DATA_FORMAT = "jpg"
upper_limit = upper_limit_ALT # when no decimal point is use set to ALT

class_distribution = np.zeros(len(names))

if make_test_train:
    try:
        mkdir(join(output_location,"train"))
        mkdir(join(output_location,"test"))
    except FileExistsError:
        pass
    except FileNotFoundError:
        print("OUTPUT LOCATION DOES NOT EXIST")

for name in names:
    try:
        if make_test_train:
            mkdir(join(output_location,"train",name))
            mkdir(join(output_location,"test",name))
        else:
            mkdir(join(output_location,name))
    except FileExistsError:
        pass
    except FileNotFoundError:
        print("OUTPUT LOCATION DOES NOT EXIST")
        break

for f in pbar(listdir(dataset_location)):
    if isfile(join(dataset_location, f)):
        if f[-3:] == DATA_FORMAT:
            class_name = f.split(SPLIT_KEY)[-1][:-4]
            class_value = float(class_name)
            if USE_CLASS_LIMITS:
                #print(class_value)
                found_class = False
                frame_num = int(f.split(SPLIT_KEY)[-2])
                for c, cv in enumerate(upper_limit):
                    if class_value <= cv:
                        if DATA_FORMAT != OUTPUT_DATA_FORMAT:
                            img = cv2.imread(join(dataset_location,f))
                        if make_test_train:
                            if frame_num < num_train + startframe:
                                if DATA_FORMAT != OUTPUT_DATA_FORMAT:
                                    cv2.imwrite(join(output_location,"train",names[c],f[:-3] + OUTPUT_DATA_FORMAT), img)
                                else:
                                    shutil.copyfile(join(dataset_location,f),join(output_location,"train",names[c],f))
                            else:
                                if DATA_FORMAT != OUTPUT_DATA_FORMAT:
                                    cv2.imwrite(join(output_location,"test",names[c],f[:-3] + OUTPUT_DATA_FORMAT), img)
                                else:
                                    shutil.copyfile(join(dataset_location,f),join(output_location,"test",names[c],f))
                        else:
                            cv2.imwrite(join(output_location,names[c],f[:-3] + OUTPUT_DATA_FORMAT), img)
                        # shutil.copyfile(join(dataset_location,f), join(output_location,names[c],f))
                        found_class = True
                        class_distribution[c] += 1
                        break
                # put file into unbounded class if larger than any class defined in upper_limit
                if not found_class:
                    if make_test_train:
                        if frame_num < num_train + startframe:
                            if DATA_FORMAT != OUTPUT_DATA_FORMAT:
                                img = cv2.imread(join(dataset_location,f))
                                cv2.imwrite(join(output_location,"train",names[-1],f[:-3] + OUTPUT_DATA_FORMAT), img)
                            else:
                                shutil.copyfile(join(dataset_location,f),join(output_location,"train",names[-1],f))
                        else:
                            if DATA_FORMAT != OUTPUT_DATA_FORMAT:
                                img = cv2.imread(join(dataset_location,f))
                                cv2.imwrite(join(output_location,"test",names[-1],f[:-3] + OUTPUT_DATA_FORMAT), img)
                            else:
                                shutil.copyfile(join(dataset_location,f),join(output_location,"test",names[-1],f))
                    else:
                        img = cv2.imread(join(dataset_location,f))
                        cv2.imwrite(join(output_location,names[-1],f[:-3] + OUTPUT_DATA_FORMAT), img)
                        #shutil.copyfile(join(dataset_location,f), join(output_location,names[-1],f))
                    class_distribution[-1] += 1
            else:
                shutil.copyfile(join(dataset_location,f), join(output_location,class_name,f))

In [ ]:
plt.title("Class distribution of parsed dataset")             
plt.xticks(ticks=range(len(class_distribution)),
           labels=names,
           rotation=45)
plt.xlabel("Class")
plt.ylabel("number of samples")
plt.bar(x=range(len(class_distribution)),
        height=class_distribution)
plt.show()

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  output_location,
  image_size=(128, 128),
  batch_size=32)

In [ ]:
train_ds?

In [ ]:
def make_gauss_pdf(y,sigma=1,sum_to_one=True):
    """
    y : label vector (one-hot encoded) with length n classes
    sigma : standard deviation of probability density function

    the distribution will be centered around the maximum activation
    which will subsequently treated as the dsitribution mean
    """
    y_gauss = np.zeros(len(y))
    mu = np.argmax(y)
    for c in range(len(y)):
        y_gauss[c] = (1/(sigma * np.sqrt(2*np.pi))) * np.exp((-1/2) * np.square((c-mu)/sigma))

    # to use softmax for the final activation layer the sum of the PDF should be equal to one
    if sum_to_one:
        mult_fact = np.sum(y_gauss)
        y_gauss = y_gauss / mult_fact
    
    return y_gauss

# example one-hot encoded groundtruth vector
y_test = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]
y_gauss = make_gauss_pdf(y_test,sigma=2)

# sigma can be treated as a tuneable parameter for regularisation to convey relationships between
# adjacent classes and surpress early over-confidence during training. A wrong prediction close to
# the target class can thus be penalised less than a wrong prediction further from the true class.

plt.plot(y_test,'bo')
plt.plot(y_gauss,'ro')
plt.title("Example activation of the classificaition output layer")
plt.legend(["one-hot encoding","gauss PDF"])
plt.show()